<a href="https://colab.research.google.com/github/Viny2030/polars/blob/main/hvplot_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Easiest Way to Create an Interactive Dashboard in Python

This notebook supports the blog post

**The Easiest Way to Create an Interactive Dashboard in Python. Turn Pandas pipelines into a
dashboard using hvplot `.interactive`**

by *Sophia Yang* and *Marc Skov Madsen*.

![Data App](https://github.com/sophiamyang/hvplot_interactive/blob/HEAD/assets/easy-dataframe-dashboards.gif?raw=1)

Source: https://github.com/sophiamyang/hvplot_interactive

La forma más sencilla de crear un panel interactivo en Python
Este cuaderno respalda la publicación del blog

La forma más sencilla de crear un panel interactivo en Python. Convierta las canalizaciones de Pandas en un panel con hvplot .interactive

por Sophia Yang y Marc Skov Madsen.

## Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot`.

Importar y configurar paquetes
Tenga en cuenta que en Colab deberá instalar !pip panel hvplot.

[ ]

In [1]:
# !pip install panel==0.12.6 hvplot==0.7.3

In [3]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [4]:
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

In [6]:
!pip install hvplot
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 4.0 MB/s eta 0:00:00


## Define function to determine environment

In [7]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

'notebook'

## Define Color Palette

In [9]:
import panel as pn

PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0"]

# Create colored spacers using pn.pane.HTML
# No need to wrap with HSpacer, as HTML panes can be placed directly in the Row/Column
spacer1 = pn.pane.HTML(f'<div style="background-color:{PALETTE[0]}; height:50px; width:100%"></div>')
spacer2 = pn.pane.HTML(f'<div style="background-color:{PALETTE[1]}; height:50px; width:100%"></div>')
spacer3 = pn.pane.HTML(f'<div style="background-color:{PALETTE[2]}; height:50px; width:100%"></div>')

# Arrange the spacers horizontally using pn.Row
pn.Row(spacer1, spacer2, spacer3)

Row(sizing_mode='stretch_width')
    [0] HTML(str, sizing_mode='stretch_width')
    [1] HTML(str, sizing_mode='stretch_width')
    [2] HTML(str, sizing_mode='stretch_width')

## Load Data

In [10]:
from bokeh.sampledata.autompg import autompg_clean as df
df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,North America,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,North America,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,North America,ford torino,ford


## Define DataFrame Pipeline

In [11]:
(
    df[
        (df.cyl == 4) &
        (df.mfr.isin(['ford','chevrolet']))
    ]
    .groupby(['origin', 'cyl', 'mfr', 'yr'])['hp'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='yr')
).head(1)

,origin,cyl,mfr,yr,hp
0,North America,4,chevrolet,71,81.0


## Make DataFrame Pipeline Interactive

In [12]:
idf = df.interactive()

Define [Panel widgets](https://panel.holoviz.org/reference/index.html#widgets)

In [13]:
cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
mfr = pn.widgets.ToggleGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    button_type='success')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['hp', 'weight'],
    button_type='success'
)

Combine pipeline and widgets

In [14]:
ipipeline = (
    idf[
        (idf.cyl == cylinders) &
        (idf.mfr.isin(mfr))
    ]
    .groupby(['origin', 'mpg'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='mpg')
    .reset_index(drop=True)
)
ipipeline.head()

## Pipe to Table

In [15]:
if environment()=="server":
   theme="fast"
else:
   theme="simple"

In [16]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)
itable

Check out the [Tabulator Reference Guide](https://panel.holoviz.org/reference/widgets/Tabulator.html) for more inspiration.

## Pipe to hvplot

In [17]:
ihvplot = ipipeline.hvplot(x='mpg', y=yaxis, by='origin', color=PALETTE, line_width=6, height=400)
ihvplot

## Layout using Template

Here we use the [FastListTemplate](https://panel.holoviz.org/reference/templates/FastListTemplate.html#templates-gallery-fastlisttemplate).

In [18]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive',
    sidebar=[cylinders, 'Manufacturers', mfr, 'Y axis' , yaxis],
    main=[ihvplot.panel(), itable.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

Please note that to get the Tabulator table styled nicely for dark mode you can set `theme='fast'` when you define the `itable`. It won't work in the notebook though.

To *serve the notebook* run `panel serve hvplot_interactive.ipynb`.


Tenga en cuenta que para que la tabla Tabulator tenga un estilo adecuado para el modo oscuro, puede configurar theme='fast' cuando defina el itable. Sin embargo, no funcionará en el cuaderno.

Para servir al cuaderno, ejecute panel serve hvplot_interactive.ipynb.